In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from IPython.display import Latex
from tqdm import tqdm

from sklearn import metrics
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import KFold

import numpy as np
import pandas as pd
import scipy as sp
import statistics

# set manual seeds for reproducible results
torch.manual_seed(123)
np.random.seed(123)

In [2]:
# GPU / CPU selection
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"

# device = "cpu"

print(device)

cuda:0


In [3]:
# dataset loading function
def load_breast_cancer_data():
    dbName = 'breast_cancer'
    y_column_array = ['target']
    data = load_breast_cancer()
    df = pd.DataFrame(data.data, columns=data.feature_names)
    df_clean = df.dropna()
    y_actual = data.target
    
    return df_clean.to_numpy(), y_actual


In [4]:
class Autoencoder(nn.Module):
    
    def __init__(self,input_level,output_level):
        
        super().__init__()        
        self.encoder = nn.Sequential(
            nn.Linear(input_level, output_level),
            nn.ReLU()
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(output_level, input_level),
            nn.ReLU()
        )
        

    def forward(self, x):
        
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded

In [5]:
class Deep_Autoencoder(nn.Module):
    
    def __init__(self,input_level, output_level):
        
        super().__init__()
        
        level_1 = 500
        level_2 = 500
        level_3 = 2000
        
        self.encoder = nn.Sequential(
            nn.Linear(input_level, level_1),
            nn.ReLU(),
            nn.Linear(level_1, level_2),
            nn.ReLU(),
            nn.Linear(level_2, level_3),
            nn.ReLU(),
            nn.Linear(level_3, output_level),
            nn.ReLU()
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(output_level, level_3),
            nn.ReLU(),
            nn.Linear(level_3, level_2),
            nn.ReLU(),
            nn.Linear(level_2, level_1),
            nn.ReLU(),
            nn.Linear(level_1, input_level),
            nn.ReLU()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        
        
        return decoded, encoded


In [6]:
def softmax(x):
    return (np.exp (x) /np.sum (np.exp (x))).tolist()

In [7]:
%%latex
KL Divergence Loss equations for reference

\begin{eqnarray}
\\
p_{ij} =  \frac{\exp ({-||x_i - x_j||^2/2 \sigma^2})}{\sum_{k\neq l}\exp ({-||x_k - x_l||^2/2 \sigma^2})}\\
q_{ij} =  \frac{\exp ({-||z_i - z_j||^2/2 \sigma^2}}{\sum_{k\neq l}\exp ({-||z_k - z_l||^2/2 \sigma^2})}\\
KL~(P || Q) = \sum_i \sum_j p_{ij} log \frac{p_{ij}}{q_{ij}}\\
\\
\end{eqnarray}

<IPython.core.display.Latex object>

In [8]:
def mahalanobis(x=None, centroid=None):
    """Compute the Mahalanobis Distance between each row of x and the data  
    x    : vector or matrix of data with, say, p columns.
    data : ndarray of the distribution from which Mahalanobis distance of each observation of x is to be computed.
    cov  : covariance matrix (p x p) of the distribution. If None, will be computed from data.
    """
    x_minus_mu = x - centroid
    cov = np.cov(np.transpose(x))
    inv_covmat = sp.linalg.pinv(cov)
    left_term = np.dot(x_minus_mu, inv_covmat)
    mahal = np.dot(left_term, x_minus_mu.T)
    return mahal.diagonal()

In [9]:
def KLDiv_p(X, n_clusters):
    EPS = 1e-12
    
    n_samples = X.shape[0]
    
    # find cluster labels of x using gmm or kmeans
    if cluster_method == 'kmeans':
        kmeans_x = KMeans(n_clusters = n_clusters, init='k-means++',random_state=123)
        labels_x = kmeans_x.fit_predict(X)
    else:
        # replacing with gmm.cluster_centers_ won't work
        gmm_x = GaussianMixture(n_components=n_clusters, random_state=123)
        labels_x = gmm_x.fit_predict(X)
    
    # calculate cluster centers  
    x_centroids = []
    for j in range(n_clusters):
        cluster_samples_x = X[labels_x == j]
        cluster_centroid_x = np.mean(cluster_samples_x, axis = 0)
        x_centroids.append(cluster_centroid_x)
    
    x_centroids = np.array(x_centroids)
    
    
    # calculate mahalanobis distance of samples from cluster center
    mahal_p = []
    for j in range(n_clusters):
        mahal_p.append(mahalanobis(X, x_centroids[j]))
    
    # calculate p numerator and denominator
    mahal_p = np.array(mahal_p)
    mahal_p = mahal_p * -1
    p_numerator = np.exp (np.add(mahal_p,EPS))
    p_denominator = np.sum (p_numerator, axis=0)
            
    # p is the target distribution
    p = np.zeros([n_samples, n_clusters])
    for i in range(n_samples):
        for j in range(n_clusters):
            p[i][j] = p_numerator[j][i] / p_denominator[i]
            
    return p

In [10]:
def KLDiv_loss(latent, n_clusters, p):
    EPS = 1e-12

    n_samples = latent.shape[0]
    
    # find cluster labels of z using gmm or kmeans
    if cluster_method == 'kmeans':
        kmeans_z = KMeans(n_clusters = n_clusters, init='k-means++',random_state=123)
        labels_z = kmeans_z.fit_predict(latent)
    else:
        gmm_z = GaussianMixture(n_components=n_clusters, random_state=123)
        labels_z = gmm_z.fit_predict(latent)
        
    # calculate cluster centers    
    z_centroids = []
    for j in range(n_clusters):    
        cluster_samples_z = latent[labels_z == j]
        cluster_centroid_z = np.mean(cluster_samples_z, axis = 0)
        z_centroids.append(cluster_centroid_z)
    
    z_centroids = np.array(z_centroids)
    
    # calculate mahalanobis distance of samples from cluster center
    mahal_q = []
    for j in range(n_clusters):
        mahal_q.append(mahalanobis(latent, z_centroids[j]))
    
    # calculate q numerator and denominator
    mahal_q = np.array(mahal_q)
    mahal_q = mahal_q * -1
    q_numerator = np.exp (np.add(mahal_q,EPS))
    q_denominator = np.sum (q_numerator, axis=0)

    # q is the predicted distribtuion
    q = np.zeros([n_samples, n_clusters])
    for i in range(n_samples):
        for j in range(n_clusters):
            q[i][j] = q_numerator[j][i] / q_denominator[i]

    # calculate KL div
    K_L_div = 0
    for i in range(q.shape[0]):
        row_sum = 0
        for j in range(q.shape[1]):
            row_sum += p[i][j] * np.log(p[i][j] /q[i][j])
        K_L_div+= row_sum
    K_L_div = K_L_div / (n_samples * n_clusters)
    
    return K_L_div, q

In [11]:
def training_and_testing(dataset, latent_dim, gamma_epoch, l_rate, result_to_csv):
    
    acc_folds= []
    nmi_folds = []
    ari_folds = []
    fold_counter = 0
    
    X, y_actual = load_breast_cancer_data()
    no_of_clusters = len(np.unique(y_actual))
    
    kfold = KFold(n_splits=5, random_state=123, shuffle=True)
        
    # gamma calculation loop
    for gamma_index, test_index in kfold.split(X):
        
        # initialize variables at the start of fold
        fold_counter +=1
        best_gamma = 0
        max_acc = 0
        
        print("fold "+str(fold_counter))
        
        # separate train and test data
        X_test, X_gamma = X[test_index], X[gamma_index]
        y_test, y_gamma = y_actual[test_index], y_actual[gamma_index]

        # standardize data
        scaler = preprocessing.StandardScaler()
        X_gamma = scaler.fit_transform(X_gamma)
        X_test = scaler.transform(X_test)

        data_tensor = torch.from_numpy(X_gamma).float()
        
        # search for optimal gamma for each fold
        gamma_range = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
        pbar = tqdm(gamma_range)
        # --------------------training section--------------------------
        for gamma in pbar:
            
            ae_loss = []
            kl_loss = []
            ae_kl_loss = []

            # initialize model
            model = Deep_Autoencoder(input_level = data_tensor.size()[1], output_level = latent_dim)
            criterion = nn.MSELoss()
            optimizer = torch.optim.Adam(model.parameters(), lr = l_rate)

            # to run code on gpu
            model = model.to(device)
            data_tensor = data_tensor.to(device)

            # pre-calculate p outside the loop once as it doesn't change over epochs
            p = KLDiv_p(X = X_gamma, n_clusters = no_of_clusters)
            
            # train for 5000 epochs
            for epoch in range(gamma_epoch):

                recon, latent = model(data_tensor) 
                latent_numpy = latent.cpu().detach().numpy()
                #calculate q and kld loss inside the training loop
                K_L_div, q = KLDiv_loss(latent = latent_numpy, n_clusters = no_of_clusters, p=p)        

                # calculate combined loss using reconstruction loss and KL divergence loss
                AE_loss = criterion(recon, data_tensor) 
                loss = AE_loss + gamma * K_L_div

                ae_loss.append(AE_loss.cpu().detach().numpy())
                kl_loss.append(gamma * K_L_div)
                ae_kl_loss.append(loss.item())


                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                y_pred = q.argmax(1)
                
                # calculate metrics
                gamma_acc = np.round(metrics.accuracy_score(y_gamma, y_pred), 4) *100
                gamma_nmi = np.round(metrics.normalized_mutual_info_score(y_gamma, y_pred), 4) *100
                gamma_ari = np.round(metrics.adjusted_rand_score(y_gamma, y_pred), 4) *100

                # save the model with the best accuracy
                if gamma_acc > max_acc:
                    max_acc =  gamma_acc
                    iter_num_max_acc = epoch
                    best_gamma = gamma
                    torch.save(model.state_dict(), 'Results/' + dataset + '_pretrained_best_model.pt')
                    
                pbar.set_description(f'gamma={gamma} (best={best_gamma}); acc={gamma_acc:.3f} (best={max_acc:.3f}); epoch={epoch}, loss={loss.item():.3f}')

           
        # testing
        # initialize and load pretrained model
        model = Deep_Autoencoder(input_level = data_tensor.size()[1], output_level = latent_dim)
        model.load_state_dict(torch.load('Results/' + dataset + '_pretrained_best_model.pt'))

        # convert test data to tensor
        data_tensor = torch.from_numpy(X_test).float()

        # load model and data on device(cpu/gpu)
        model = model.to(device)
        data_tensor = data_tensor.to(device)

        # run model
        recon, latent = model(data_tensor)
        latent_numpy = latent.cpu().detach().numpy()
        
        # get test predictions
        p = KLDiv_p(X = X_test, n_clusters = no_of_clusters)
        K_L_div, q = KLDiv_loss(latent = latent_numpy, n_clusters = no_of_clusters, p = p)
        y_pred = q.argmax(1)

        # calculate metrics
        current_acc = np.round(metrics.accuracy_score(y_test, y_pred), 4) *100
        current_nmi = np.round(metrics.normalized_mutual_info_score(y_test, y_pred), 4) *100
        current_ari = np.round(metrics.adjusted_rand_score(y_test, y_pred), 4) *100
        
        acc_folds.append(current_acc)
        nmi_folds.append(current_nmi)
        ari_folds.append(current_ari)

        # Save results in csv
        result_to_csv.append(
            [
                dataset,
                latent_dim,
                best_gamma,
                current_acc,
                current_nmi,
                current_ari,
                iter_num_max_acc

            ]
        )
        

    # calculate mean and standard deviation of five folds
    result_to_csv.append([
        dataset,
        latent_dim,
        '-',
        str(np.round(statistics.mean(acc_folds), 2)) + " (" + str(np.round(statistics.pstdev(acc_folds), 2))+")",
        str(np.round(statistics.mean(nmi_folds), 2)) + " (" + str(np.round(statistics.pstdev(nmi_folds), 2))+")",
        str(np.round(statistics.mean(ari_folds), 2)) + " (" + str(np.round(statistics.pstdev(ari_folds), 2))+")"
    ])
    
    return result_to_csv

In [ ]:
dataset_list = ['breast_cancer']
learning_rate = 1e-4
cluster_method = 'gmm' # 'gmm' or 'kmeans'
latent_dim = 10
number_of_epoch = 5000

for dataset in dataset_list:
    print(dataset)
    result_to_csv = []
    result_to_csv.append(
        [
            "Dataset",
            "dimension",
            "gamma",
            "Accuracy",
            "NMI",
            "ARI",
            "iter_num_max_acc"
        ]
    )

    result_to_csv = training_and_testing(dataset = dataset , latent_dim = latent_dim,
                                            gamma_epoch = 5000, l_rate = learning_rate,
                                            result_to_csv=result_to_csv)
        

        

    pd.DataFrame(result_to_csv).to_csv("Results/gceals_"+ dataset +".csv")